# Figures from Water Column Model Runs

In [3]:
# Data Science and Visualization Imports
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import xarray as xr
import numpy as np

### Data Imports

In [4]:
wc_results_path = 'results_data/wc_runs_all.nc'
input_data_path = 'results_data/usgs_data.csv'
wc_ds = xr.open_dataset(wc_results_path)
df = pd.read_csv(input_data_path)

Compiling Data from modeling runs for analysis

In [5]:
df_res = pd.DataFrame({
    'mean_u': ([vel.values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_25': ([vel[:20].values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_50': ([vel[:40].values.mean() for vel in wc_ds['U Velocity']]),
    'mean_u_75': ([vel[:60].values.mean() for vel in wc_ds['U Velocity']]),
    'Q2': ([vel.values.max() for vel in wc_ds['Q2']]),
    'wc_height': df['wc_height'],
    'avg_ht': df['ave_ht'],
    'density_total': df['density_final'],
    'frontal_total': df['frontal_total'],
    'site': df['site'],
})
df_res.head()

,mean_u,mean_u_25,mean_u_50,mean_u_75,Q2,wc_height,avg_ht,density_total,frontal_total,site
0,-1.368986,-0.375091,-0.830812,-1.139095,0.243219,0.6759,6.5,630.481094,1103.449997,Bay
1,-0.513566,-0.006930,-0.180808,-0.361371,0.127301,0.4855,11.0,2837.164924,8403.196133,Bay
2,-0.336038,-0.000537,-0.064772,-0.203614,0.117241,0.5651,19.0,3467.646018,17740.080726,Bay
3,-0.457094,-0.000490,-0.132418,-0.305199,0.135983,0.5655,15.0,4728.608206,19098.173030,Bay
4,-0.686641,-0.036265,-0.285306,-0.506819,0.167223,0.5855,11.0,3467.646018,10270.573052,Bay


## EDA and Scatterplots

In [6]:
fig = px.scatter(df_res, x='Q2', y='avg_ht', trendline='ols', symbol='site', color='site')
fig.show()

In [7]:
fig = px.scatter(df_res, x='mean_u', y='wc_height', trendline='ols', symbol='site', color='site')
fig.show()

In [8]:
fig = px.scatter(df_res, x='mean_u_25', y='density_total', trendline='ols', symbol='site', color='site')
fig.show()

In [9]:
fig = px.scatter(df_res, x='mean_u', y='mean_u_25', symbol='site', color='site')
fig.show()

## Contour Plots

## Maps

## Animations